# Pruebas con Optuna

### Colab

In [1]:
import os
import sys
import IPython

# Detectar si estamos en Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Ruta base
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/ia_thermal_colab"
else:
    BASE_PATH = os.path.expanduser("~/ia_thermal_colab")

DATASETS_PATH = os.path.join(BASE_PATH, "datasets")
MODELS_PATH = os.path.join(BASE_PATH, "models")

os.makedirs(DATASETS_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

print("Modo:", "Colab" if IN_COLAB else "Local")
print("Ruta datasets:", DATASETS_PATH)
print("Ruta modelos:", MODELS_PATH)

Mounted at /content/drive
Modo: Colab
Ruta datasets: /content/drive/MyDrive/ia_thermal_colab/datasets
Ruta modelos: /content/drive/MyDrive/ia_thermal_colab/models


In [2]:
# 🔄 Parámetros del repo
GIT_REPO_URL = "https://github.com/ismaelgallolopez/ia_thermal.git"  # 👈 Cambia esto
REPO_NAME = GIT_REPO_URL.split("/")[-1].replace(".git", "")
CLONE_PATH = os.path.join(BASE_PATH, REPO_NAME)

if IN_COLAB:
    # 🧬 Clonar el repositorio si no existe ya
    if not os.path.exists(CLONE_PATH):
        !git clone {GIT_REPO_URL} {CLONE_PATH}
    else:
        print(f"Repositorio ya clonado en: {CLONE_PATH}")

    # 📦 Instalar requirements.txt
    req_path = os.path.join(CLONE_PATH, "requirements.txt")
    if os.path.exists(req_path):
        !pip install -r {req_path}
    else:
        print("No se encontró requirements.txt en el repositorio.")

    print("🔄 Reinicia el entorno para aplicar los cambios...")
    IPython.display.display(IPython.display.Javascript('''google.colab.restartRuntime()'''))

Repositorio ya clonado en: /content/drive/MyDrive/ia_thermal_colab/ia_thermal
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

<IPython.core.display.Javascript object>

## Inicializar

In [15]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# get the directory path of the file
dir_path = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal")

from plot_functions import *
from Physics_Loss import *
from utils import *
sys.path.append('../Convolutional_NN')

if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal/Convolutional_NN")

from Dataset_Class import *


if IN_COLAB:
  sys.path.append("/content/drive/MyDrive/ia_thermal_colab/ia_thermal/ismaelgallo")

from architectures.convlstm import *
from architectures.generic_spatiotemporal_decoder import *
from architectures.generic_spatiotemporal_regressor import *

if IN_COLAB:
    # Path within your Google Drive
    study_path = os.path.join(BASE_PATH, "optuna_studies")
    os.makedirs(study_path, exist_ok=True)  # Create if not exists
    storage_url = f"sqlite:///{study_path}/study_decoder.db"
else:
    os.makedirs("optuna_studies", exist_ok=True)
    storage_url = "sqlite:///optuna_studies/study_decoder.db"

In [4]:
epochs = 500
n_train = 1000
n_val = 200
sequence_length = 20

In [5]:
if IN_COLAB:
  dir_path = BASE_PATH

# ⬅️ Esto se ejecuta una vez
dataset_train = load_trimmed_dataset(
    base_path=dir_path,
    dataset_type='train',
    max_samples=n_train,
    time_steps_output=sequence_length
)
dataset_val = load_trimmed_dataset(
    base_path=dir_path,
    dataset_type='val',
    max_samples=n_val,
    time_steps_output=sequence_length
)

# Convertir a tensores de entrada y salida
input_train, output_train = prepare_data_for_convlstm(dataset_train, device='cpu')
input_val, output_val = prepare_data_for_convlstm(dataset_val, device='cpu')


def get_data_loaders_from_tensors(batch_size):
    train_loader = DataLoader(TensorDataset(input_train, output_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(input_val, output_val), batch_size=batch_size, shuffle=False)
    return train_loader, val_loader


✅ Cargando dataset train desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset_train.pth


/content/drive/MyDrive/ia_thermal_colab/ia_thermal/Convolutional_NN/Dataset_Class.py:299: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_dataset = torch.load(full_path)


✅ Cargando dataset val desde: /content/drive/MyDrive/ia_thermal_colab/datasets/PCB_transient_dataset_val.pth


## ConvLSTM

In [6]:
class ConvLSTMWrapper(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers):
        super().__init__()
        self.convlstm = ConvLSTM(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            kernel_size=kernel_size,
            num_layers=num_layers,
            batch_first=True,
            bias=True,
            return_all_layers=False
        )

    def forward(self, x):
        output, _ = self.convlstm(x)  # output es lista [layer_output]
        return output[0]              # devolvemos directamente el tensor (B, T, C, H, W)


In [7]:
def objective(trial):
    # Hiperparámetros
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    hidden_dim_val = trial.suggest_categorical("hidden_dim", [16, 32, 64, 128])
    kernel_size_val = trial.suggest_categorical("kernel_size", [1, 3, 5, 7])
    num_layers = trial.suggest_int("num_layers", 1, 4)

    # Adaptar al formato requerido por ConvLSTM
    hidden_dim = [hidden_dim_val] * num_layers
    kernel_size = [(kernel_size_val, kernel_size_val)] * num_layers

    # Crear modelo ConvLSTM
    model = ConvLSTMWrapper(
        input_dim=3,  # o el número de canales reales
        hidden_dim=hidden_dim,
        kernel_size=kernel_size,
        num_layers=num_layers
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    criterion = torch.nn.MSELoss()

    # Obtener dataloaders (train y val)
    train_loader, val_loader = get_data_loaders_from_tensors(batch_size=batch_size)

    best_val_loss = float('inf')
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss


In [8]:
study = optuna.create_study(
    direction="minimize",
    study_name="nombre_del_estudio",  # puedes elegirlo tú
    storage="sqlite:///optuna_studies/study_convlstm.db",  # 🧠 crea archivo .db en tu carpeta
    load_if_exists=True  # por si ya existe, continúa donde lo dejaste
)

study.optimize(objective, n_trials=100)

[I 2025-05-06 06:34:02,490] A new study created in memory with name: no-name-194fc0af-b824-4817-9543-9d005052578a
Epoch 1/500 - Training:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 20, 1, 13, 13])) that is different to the input size (torch.Size([32, 20, 32, 13, 13])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/500 - Training:  94%|█████████▍| 30/32 [00:01<00:00, 21.68it/s, loss=0.0753]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 20, 1, 13, 13])) that is different to the input size (torch.Size([8, 20, 32, 13, 13])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reducti

KeyboardInterrupt: 

In [ ]:
print("Best trial:")
for key, val in study.best_trial.params.items():
    print(f"{key}: {val}")

In [ ]:
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_param_importances(study).show()


## Decoder

In [16]:
def objective(trial):
    # Hiperparámetros estructurales
    embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 1, 6)
    nhead = trial.suggest_categorical("nhead", [1, 2, 4, 8])
    dim_feedforward_factor = trial.suggest_int("dim_ff_factor", 2, 6)
    use_temporal_channel = trial.suggest_categorical("use_temporal_channel", [False, True])

    # Hiperparámetros de entrenamiento
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)

    # Elegir optimizador
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop"])

    # Dataloaders
    train_loader, val_loader = get_data_loaders_from_tensors(batch_size)

    # Modelo
    class CustomTransformerDecoder(TransformerDecoder):
        def __init__(self, embedding_dim, num_layers, nhead, dim_ff, dropout):
            super().__init__(embedding_dim, num_layers, nhead)
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=embedding_dim,
                nhead=nhead,
                dim_feedforward=dim_ff,
                dropout=dropout,
                batch_first=True
            )
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    model = GenericSpatioTemporalDecoder(
        embedding_dim=embedding_dim,
        num_layers=num_layers,
        nhead=nhead,
        in_channels=3,
        use_temporal_channel=use_temporal_channel
    ).to(device)

    # Reemplazar el decoder por uno con dropout y tamaño FF ajustado
    model.temporal_decoder = CustomTransformerDecoder(
        embedding_dim=embedding_dim,
        num_layers=num_layers,
        nhead=nhead,
        dim_ff=embedding_dim * dim_feedforward_factor,
        dropout=dropout
    ).to(device)

    # Optimizador
    optimizer_cls = {"Adam": torch.optim.Adam, "AdamW": torch.optim.AdamW, "RMSprop": torch.optim.RMSprop}[optimizer_name]
    optimizer = optimizer_cls(model.parameters(), lr=lr, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
    criterion = torch.nn.MSELoss()

    # Entrenamiento
    best_val_loss = float("inf")
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss


In [ ]:
study = optuna.create_study(
    direction="minimize",
    study_name="decoder1",  # puedes elegirlo tú
    storage=storage_url,  # 🧠 crea archivo .db en tu carpeta
    load_if_exists=True  # por si ya existe, continúa donde lo dejaste
)

study.optimize(objective, n_trials=100)

# Mostrar el mejor resultado
print("🔍 Best trial:")
for k, v in study.best_trial.params.items():
    print(f"{k}: {v}")


[I 2025-05-06 07:36:38,948] A new study created in RDB with name: decoder1
[I 2025-05-06 07:54:32,385] Trial 0 finished with value: 0.021076383498998787 and parameters: {'embedding_dim': 512, 'num_layers': 1, 'nhead': 4, 'dim_ff_factor': 5, 'use_temporal_channel': True, 'batch_size': 16, 'lr': 0.002594618385880667, 'weight_decay': 2.0078385486443473e-05, 'dropout': 0.1745864308897127, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.021076383498998787.
[I 2025-05-06 08:10:17,864] Trial 1 finished with value: 0.0008704521460458636 and parameters: {'embedding_dim': 128, 'num_layers': 1, 'nhead': 2, 'dim_ff_factor': 4, 'use_temporal_channel': True, 'batch_size': 32, 'lr': 0.0009351820145979584, 'weight_decay': 0.0003243004235869649, 'dropout': 0.21133949920051645, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.0008704521460458636.
[I 2025-05-06 08:28:27,288] Trial 2 finished with value: 5.3424865784888554e-05 and parameters: {'embedding_dim': 512, 'num_layers': 1, 'nhead': 4, 'di

In [ ]:
import optuna.visualization as vis

vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()


## Regressor

In [ ]:
# Cargar los datos una sola vez
dataset_train = load_trimmed_dataset(
    base_path=dir_path, dataset_type='train',
    max_samples=n_train, time_steps_output=sequence_length
)
dataset_val = load_trimmed_dataset(
    base_path=dir_path, dataset_type='val',
    max_samples=n_val, time_steps_output=sequence_length
)

x_train, y_train = prepare_data_for_convlstm(dataset_train, device='cpu')
x_val, y_val = prepare_data_for_convlstm(dataset_val, device='cpu')

train_dataset = TemporalRegressionDataset(x_train, y_train)
val_dataset = TemporalRegressionDataset(x_val, y_val)


In [ ]:
def objective(trial):
    # Hiperparámetros
    embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 256])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    nhead = trial.suggest_categorical("nhead", [1, 2, 4])
    dim_ff_factor = trial.suggest_int("dim_ff_factor", 2, 6)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop"])

    # DataLoaders desde datasets pre-cargados
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Modelo
    model = GenericSpatioTemporalRegressor(
        embedding_dim=embedding_dim,
        num_layers=num_layers,
        nhead=nhead,
        in_channels=3
    ).to(device)

    # Reemplazo del decoder por uno personalizado con dropout y FF dimensionado
    class CustomTransformerDecoder(TransformerDecoder):
        def __init__(self, embedding_dim, num_layers, nhead, dim_ff, dropout):
            super().__init__(embedding_dim, num_layers, nhead)
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=embedding_dim,
                nhead=nhead,
                dim_feedforward=dim_ff,
                dropout=dropout,
                batch_first=True
            )
            self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    model.temporal_decoder = CustomTransformerDecoder(
        embedding_dim=embedding_dim * 2,
        num_layers=num_layers,
        nhead=nhead,
        dim_ff=embedding_dim * dim_ff_factor,
        dropout=dropout
    ).to(device)

    # Optimizador
    optimizer_cls = {"Adam": torch.optim.Adam, "AdamW": torch.optim.AdamW, "RMSprop": torch.optim.RMSprop}[optimizer_name]
    optimizer = optimizer_cls(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=5)
    criterion = torch.nn.MSELoss()

    best_val_loss = float("inf")
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, epochs)
        val_loss = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss


In [ ]:
study = optuna.create_study(
    direction="minimize",
    study_name="nombre_del_estudio",  # puedes elegirlo tú
    storage="sqlite:///optuna_studies/study_regressor.db",  # 🧠 crea archivo .db en tu carpeta
    load_if_exists=True  # por si ya existe, continúa donde lo dejaste
)

study.optimize(objective, n_trials=100)

# Mostrar el mejor resultado
print("🔍 Best trial:")
for k, v in study.best_trial.params.items():
    print(f"{k}: {v}")
